In [1]:
# https://keras.io/examples/vision/mnist_convnet/
# https://www.kaggle.com/code/shivanisheth2911/lstm-keras-on-mnist

In [2]:
import keras
from keras import layers
import time
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, LSTM

## Dataset

In [3]:
input_shape = (28, 28, 1)
num_classes = 10

In [4]:
# Dataset Size
# Load the data and split it between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


### Make Models

In [5]:
def make_cnn(input_shape, num_classes):
    model = keras.Sequential(
        [
            keras.Input(shape=input_shape),
            layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
            layers.MaxPooling2D(pool_size=(2, 2)),
            layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
            layers.MaxPooling2D(pool_size=(2, 2)),
            layers.Flatten(),
            layers.Dropout(0.5),
            layers.Dense(num_classes, activation="softmax"),
        ]
    )

    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

    return model

In [6]:
def make_fnn(input_shape, num_classes):
    model = keras.Sequential(
        [
            keras.Input(shape=input_shape),
            layers.Dense(num_classes, activation="relu"),
            layers.Dense(num_classes, activation="relu"),
            layers.Flatten(),
            layers.Dropout(0.5),
            layers.Dense(num_classes, activation="softmax"),
        ]
    )

    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

    return model

In [7]:
def make_rnn(input_shape, num_classes):
    model = keras.Sequential(
        [
            SimpleRNN(128, return_sequences=False,
                    input_shape=(input_shape[0], input_shape[1]),
                    unroll=True),
            layers.Dense(num_classes, activation="relu"),
            layers.Flatten(),
            layers.Dropout(0.5),
            layers.Dense(num_classes, activation="softmax"),
        ]
    )

    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

    return model

## Experiment

In [8]:
def fit_and_validate(model_function, model_name, epochs, x_train, y_train, x_test, y_test, df, input_shape, num_classes):
    model = model_function(input_shape, num_classes)
    batch_size = 128

    start_time = time.time()
    model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)
    time_to_train = time.time() - start_time

    score = model.evaluate(x_test, y_test, verbose=0)
    print("Test loss:", score[0])
    print("Test accuracy:", score[1])

    start_time = time.time()
    model.predict(x_test)
    time_to_predict = time.time() - start_time
    
    start_time = time.time()
    model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)
    time_to_retrain = time.time() - start_time

    df.loc[len(df)] = [model_name, batch_size, epochs, len(x_train), time_to_train, time_to_retrain, time_to_predict, score[1]]
    return df

In [9]:
# Action Space
all_epochs = [2]
all_models = {
    "RNN": make_rnn,
    "FNN": make_fnn,
    "CNN": make_cnn
}
all_dataset_sizes = [i/100 for i in range(25, 105, 5)]

In [10]:
# Data Collection
df = pd.DataFrame(columns=['model', 'batch_size', 'epochs', 'dataset_size', 'time_to_train', 'time_to_retrain', 'time_to_predict', 'accuracy'])

In [11]:
for i in range(10):
    for epochs in all_epochs:
        for model in all_models:
            for dataset_size in all_dataset_sizes:
                print(f"{i}/10  {epochs}  {model}  {dataset_size}")
                size = int(len(x_train) * dataset_size)
    
                subset_x_train = x_train[:size]
                subset_y_train = y_train[:size]
                subset_x_test = x_test[:size]
                subset_y_test = y_test[:size]
                
                df = fit_and_validate(all_models[model], model, epochs, subset_x_train, subset_y_train, subset_x_test, subset_y_test, df, input_shape, num_classes)
                df.to_csv("results.csv")

0/10  2  RNN  0.25
Epoch 1/2


C:\Users\ajrbe\Documents\Git\deeplearning-performance-evaluation\.venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


106/106 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.2305 - loss: 2.0049 - val_accuracy: 0.6793 - val_loss: 1.1647
Epoch 2/2
106/106 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.4608 - loss: 1.4173 - val_accuracy: 0.7053 - val_loss: 0.9269
Test loss: 0.834019124507904
Test accuracy: 0.7541000247001648
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step   
Epoch 1/2
106/106 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.5091 - loss: 1.2564 - val_accuracy: 0.7713 - val_loss: 0.8094
Epoch 2/2
106/106 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.5311 - loss: 1.1922 - val_accuracy: 0.8107 - val_loss: 0.7257
0/10  2  RNN  0.3
Epoch 1/2


C:\Users\ajrbe\Documents\Git\deeplearning-performance-evaluation\.venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


127/127 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.2290 - loss: 2.0369 - val_accuracy: 0.7394 - val_loss: 1.1278
Epoch 2/2
127/127 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.4746 - loss: 1.4166 - val_accuracy: 0.8561 - val_loss: 0.7565
Test loss: 0.7285699248313904
Test accuracy: 0.8669999837875366
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step   
Epoch 1/2
127/127 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.5546 - loss: 1.1942 - val_accuracy: 0.8911 - val_loss: 0.5990
Epoch 2/2
127/127 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.6059 - loss: 1.0784 - val_accuracy: 0.8950 - val_loss: 0.5221
0/10  2  RNN  0.35
Epoch 1/2


...

380/380 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.3611 - loss: 1.6776 - val_accuracy: 0.8387 - val_loss: 0.6643
Epoch 2/2
380/380 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.5685 - loss: 1.0955 - val_accuracy: 0.8774 - val_loss: 0.4967
Test loss: 0.47091981768608093
Test accuracy: 0.8898000121116638
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step    
Epoch 1/2
380/380 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.6097 - loss: 0.9820 - val_accuracy: 0.9259 - val_loss: 0.3202
Epoch 2/2
380/380 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.6515 - loss: 0.8900 - val_accuracy: 0.9350 - val_loss: 0.2746
9/10  2  RNN  0.95
Epoch 1/2


C:\Users\ajrbe\Documents\Git\deeplearning-performance-evaluation\.venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


401/401 ━━━━━━━━━━━━━━━━━━━━ 20s 17ms/step - accuracy: 0.3946 - loss: 1.6462 - val_accuracy: 0.8840 - val_loss: 0.5346
Epoch 2/2
401/401 ━━━━━━━━━━━━━━━━━━━━ 6s 16ms/step - accuracy: 0.6043 - loss: 1.0348 - val_accuracy: 0.9058 - val_loss: 0.4185
Test loss: 0.42439696192741394
Test accuracy: 0.9060999751091003
313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step
Epoch 1/2
401/401 ━━━━━━━━━━━━━━━━━━━━ 6s 16ms/step - accuracy: 0.6357 - loss: 0.9549 - val_accuracy: 0.9379 - val_loss: 0.3154
Epoch 2/2
401/401 ━━━━━━━━━━━━━━━━━━━━ 6s 15ms/step - accuracy: 0.6573 - loss: 0.8922 - val_accuracy: 0.9382 - val_loss: 0.3001
9/10  2  RNN  1.0
Epoch 1/2


C:\Users\ajrbe\Documents\Git\deeplearning-performance-evaluation\.venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


422/422 ━━━━━━━━━━━━━━━━━━━━ 9s 16ms/step - accuracy: 0.3688 - loss: 1.6941 - val_accuracy: 0.9143 - val_loss: 0.4438
Epoch 2/2
422/422 ━━━━━━━━━━━━━━━━━━━━ 7s 15ms/step - accuracy: 0.6238 - loss: 1.0492 - val_accuracy: 0.9368 - val_loss: 0.3033
Test loss: 0.3254718780517578
Test accuracy: 0.9351000189781189
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step    
Epoch 1/2
422/422 ━━━━━━━━━━━━━━━━━━━━ 7s 16ms/step - accuracy: 0.6715 - loss: 0.9095 - val_accuracy: 0.9357 - val_loss: 0.2905
Epoch 2/2
422/422 ━━━━━━━━━━━━━━━━━━━━ 6s 15ms/step - accuracy: 0.6904 - loss: 0.8617 - val_accuracy: 0.9613 - val_loss: 0.2109
9/10  2  FNN  0.25
Epoch 1/2
106/106 ━━━━━━━━━━━━━━━━━━━━ 5s 38ms/step - accuracy: 0.6032 - loss: 1.5860 - val_accuracy: 0.8240 - val_loss: 0.5782
Epoch 2/2
106/106 ━━━━━━━━━━━━━━━━━━━━ 4s 39ms/step - accuracy: 0.8740 - loss: 0.4348 - val_accuracy: 0.8493 - val_loss: 0.4768
Test loss: 0.3534119427204132
Test accuracy: 0.8988999724388123
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
Epoch 1/2

In [12]:
df

,model,batch_size,epochs,dataset_size,time_to_train,time_to_retrain,time_to_predict,accuracy
0,RNN,128,2,15000,2.842744,1.168891,0.832688,0.7541
1,RNN,128,2,18000,2.958474,1.391817,0.781016,0.8670
2,RNN,128,2,21000,2.941556,1.594102,0.833592,0.8306
3,RNN,128,2,24000,3.167396,1.753860,0.771299,0.8234
4,RNN,128,2,27000,3.788421,2.044151,0.791324,0.9059
...,...,...,...,...,...,...,...,...
475,CNN,128,2,48000,14.735379,13.956940,1.738160,0.9806
476,CNN,128,2,51000,15.263313,14.647208,1.595118,0.9794
477,CNN,128,2,54000,16.104596,15.694815,1.981364,0.9802
478,CNN,128,2,57000,16.729838,16.223664,1.682359,0.9815
